In [1]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import dfols
import signal
from scipy.integrate import solve_ivp
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import interpolate
from stopit import threading_timeoutable as timeoutable
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from batfuns import *
plt.rcParams = set_rc_params(plt.rcParams)

eSOH_DIR = "F:/Git Hub rep/PyBaMM/Untitled Folder/data/esoh/"
oCV_DIR = "F:/Git Hub rep/PyBaMM/Untitled Folder/data/ocv/"
hppc_DIR = "F:/Stefanapolu application/RC sys ID/Peyman2020/hppc/"
crate_DIR = "F:/Stefanapolu application/RC sys ID/Peyman2020/crate/"
cyc_DIR = "F:/Stefanapolu application/RC sys ID/Peyman2020/cycling/"
fig_DIR = "../Untitled Folder/figures/figures_cycling/"
res_DIR = "../Untitled Folder/data/results/"

# %matplotlib widget

In [2]:
DFN = pybamm.lithium_ion.DFN(
    {
        # "SEI": "ec reaction limited",
        # # "loss of active material": ("stress-driven","none"),
        # "loss of active material": "stress-driven",
        # "stress-induced diffusion": "true",
        # "lithium plating": "reversible",
    }
)
# spm.print_parameter_info()
param=DFN.param

In [3]:
parameter_values = get_parameter_values()

In [4]:
cell = 1
cell_no,dfe,dfe_0,dfo_0,N,N_0 = load_data(cell,eSOH_DIR,oCV_DIR)
# eps_n_data,eps_p_data,c_rate_c,c_rate_d,dis_set,Temp,SOC_0 = init_exp(cell_no,dfe,spm,parameter_values)
# pybamm.set_logging_level("WARNING")
pybamm.set_logging_level("NOTICE")

FileNotFoundError: [Errno 2] No such file or directory: 'F:/Git Hub rep/PyBaMM/Untitled Folder/data/esoh/aging_param_cell_01.csv'

In [ ]:
Temp=25
parameter_values = get_parameter_values()
parameter_values.update(
    {
        "Positive electrode LAM constant proportional term [s-1]": 1.2676e-07,#2.7081e-07, #4.1759e-07,#2.7081e-07, #2.5112e-07,
        "Negative electrode LAM constant proportional term [s-1]": 1.2713e-06,#3.7576e-06, #3.2693e-06,#3.7576e-06, #2.4741e-06,
        "Positive electrode LAM constant exponential term": 1.1998,#1.2446,#1.2232,#1.1942,
        "Negative electrode LAM constant exponential term": 1.1998,#1.2446,#1.2232,#1.1942,
        "Negative electrode active material volume fraction": 0.6155,
        "Positive electrode active material volume fraction": 0.4475,
        "Initial temperature [K]": 273.15+Temp,
        "Ambient temperature [K]": 273.15+Temp,
        "SEI kinetic rate constant [m.s-1]": 1.08494281e-16,
        "EC diffusivity [m2.s-1]": 8.30909086e-19,
        "SEI growth activation energy [J.mol-1]": 1.58777981e+04,
        # "SEI kinetic rate constant [m.s-1]": 1.6827e-16,
        # "EC diffusivity [m2.s-1]": 2e-18,
        # "SEI growth activation energy [J.mol-1]": 0,
    },
    check_already_exists=False,
)

In [ ]:
experiment = pybamm.Experiment(
    [
        "Charge at "+"1.5C"+" until 4.2V", 
         
    ],
)
sim = pybamm.Simulation(DFN,experiment=experiment ,parameter_values=parameter_values, solver=pybamm.CasadiSolver(mode="safe"))
sim.solve(t_eval=[60,600,1200,1800,2400,3000] ,initial_soc =0)
solution1 = sim.solution

In [ ]:
solution1.plot([
#         "Negative electrolyte concentration [mol.m-3]",
#         "Gradient of negative electrolyte potential",
#         "Negative electrolyte potential [V]",
        "X-averaged negative particle concentration [mol.m-3]",
        "R-averaged negative particle concentration [mol.m-3]",
        "X-averaged negative particle surface concentration [mol.m-3]"

]
)

## DFN.variables.search("concentration")

In [ ]:
qddd
from scipy.io import savemat

c_n_e = solution1["Negative electrolyte concentration [mol.m-3]"].entries
t = solution1["Time [s]"].entries
eta_n_e=solution1["Negative electrolyte potential [V]"]


mdic = {"c_n_e": c_n_e, 
        "t":t,
#         "eta":eta_n_e,
       }
savemat("DFN_Cell01_halfcycle_1_5C.mat",mdic)